In [ ]:
import os
os.chdir("/work")
os.makedirs(inp_out_path, exist_ok=True)
os.chdir(inp_out_path)
!pwd


In [ ]:
import pandas as pd

def create_exact_format_csv(smiles_input, output_filename):
    """
    Create CSV with exact format including duplicate column names
    """
    
    # If input is a single string, convert to list
    if isinstance(smiles_input, str):
        smiles_list = [smiles_input]
    else:
        smiles_list = smiles_input
    
    # Write directly to file to preserve exact format
    with open(output_filename, 'w', encoding='utf-8') as f:
        # Write header exactly as specified
        f.write('smile_str,num_conformers,smile_str\n')
        
        # Write data rows
        for smiles in smiles_list:
            f.write(f'{smiles},0,{smiles}\n')
    
    print(f"CSV file '{output_filename}' created with exact format!")

create_exact_format_csv(inp_smiles, output_filename="isomeric_smi_inp.csv")

In [ ]:
import os
import subprocess
import shutil

# Config
#inp_out_path = "/work/inp_TD/"
#csv_file_name = "isomeric_smi_inp"
out_prefix = "conformers_TD"
#num_conf_TD= 200

# Generate conformers
subprocess.run([
    "python", "/app/torsional-diffusion-master/generate_confs.py",
    "--model_dir", "/app/TD-trained_models/drugs_default/",
    "--out", f"{out_prefix}.pkl",
    "--test_csv", "isomeric_smi_inp.csv",
    "--inference_steps", "20",
    "--confs_per_mol", f"{num_conf_TD}",
    "--tqdm", "--batch_size", "64", "--post_mmff"
], check=True)

# Convert to SDF
subprocess.run([
    "python", "/app/pkl_to_sdf.py",
    f"{out_prefix}.pkl",
    f"{out_prefix}.sdf"
], check=True)

# Process output
#os.chdir(inp_out_path)
shutil.move(f"{out_prefix}.sdf_0_conformers.sdf", f"{out_prefix}.sdf")

# Remove first line
with open(f"{out_prefix}.sdf", 'r') as f:
    lines = f.readlines()
with open(f"{out_prefix}.sdf", 'w') as f:
    f.writelines(lines[1:])

print("Done!")